### <center> Введение

**Очистка данных** (data cleaning) — это процесс обнаружения и удаления (или исправления) повреждённых, ложных или неинформативных записей таблицы или целой базы данных. Процесс состоит из двух этапов: **поиск** и **ликвидация** (или редактирование).

*Цель* очистки данных — избавиться от «мусора», который может помешать моделированию или исказить его результаты. Во многих задачах очистка данных — это *самая главная часть* этапа подготовки данных к построению модели, которая нередко занимает большую часть времени работы над задачей.

В среднем при работе с данными очистка данных занимает около 40-60 % от общего времени работы с данными (включая построение модели).

Вспомните поговорку: **«мусор на входе — мусор на выходе».**

Данные почти всегда «грязные». В бизнес-задачах данные чаще всего представляют собой неструктурированный набор таблиц, в которых содержится куча всевозможного «мусора». 

Нам понадобятся данные из многих отделов, и все они будут представлены разными форматами: данные об активности пользователей на сайте — в виде *json*-формата, для данных из бухгалтерии типичен формат *excel*, данные о показателях развития мы достанем из *базы данных*. 

На руках у нас будет куча таблиц в разных форматах, которые необходимо грамотно объединить.

Для того чтобы «состряпать» большой и полезный датасет, необходимо отдельно обработать каждую из таблиц. В данных будет огромное количество «мусора»: начиная с несовпадающих периодов наблюдений (компания открылась раньше, чем её сайт), наименований категорий (различные названия отделов в каждой из таблиц) и пропущенных данных (мы могли потерять их в процессе выгрузки из базы), заканчивая некорректно введёнными значениями с парой лишних или недостающих нулей, которые по случайности вбила условная бухгалтер Алевтина Петровна